## Creación de un simple ChatBot que interactúa con llama 3.2

### 1. Librerías

In [4]:
!pip3 install langchain==0.3.9 langchain-core==0.3.21 langchain-ollama==0.2.0

In [5]:
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_ollama import ChatOllama
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

## 2. Large language model

Hay diferentes modelos que se pueden usar. La mayoría necesita un "API KEY":

ChatOpenAI, ChatAnthropic, ChatVertexAI, ChatCohere, ChatNVIDIA, ChatGroq, ChatMistralAI

Existen otros modelos en la plataforma HUGGINGFACE.

Para la mayoría de los modelos hay que definir una variable de entorno:

------------------------------------------------------------------------------
import getpass

import os

export OPENAI_API_KEY="..."

os.environ["OPENAI_API_KEY"] = getpass.getpass()

from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4")

------------------------------------------------------------------------------

Usando Ollama en local no es necesario definir variables de entorno. 


In [12]:
local_llm = 'llama3.2'
llm = ChatOllama(model=local_llm, temperature=0)

Algunos de los parámetros más importantes que se deben/pueden especificar:

-**model**: el nombre del modelo específico que se quiere usar (por ejemplo, para ChatOpenAI puede ser "gpt-3.5-turbo" o "gpt-4")

**temperature**: controla la aleatoriedad de la respuesta (y la capacidad "generativa"), el valor mínimo es 0 (muy baja aleatoriedad y creatividad).

**timeout**: el máximo tiempo de espera para obtener la respuesta

**max_tokens**: es un número que limita el valor máximo de palabra y puntuación en la respuesta.

**api_key**: el api key del usuario

No todos los modelos admiten los mismos parámetros.

Métodos clave del modelo de chat:

-**invoke**: El método principal para interactuar con un modelo de chat. Acepta una lista de mensajes como entrada y devuelve una lista de mensajes como salida.

-**stream**: Un método que permite transmitir la salida de un modelo de chat a medida que se genera.

-**batch**: Un método que permite agrupar varias solicitudes a un modelo de chat para su procesamiento eficiente.

-**bind_tools**: Un método que permite vincular una herramienta a un modelo de chat para su uso en el contexto de ejecución del modelo.

-**with_structured_output**: Un envoltorio alrededor del método invoke para modelos que admiten natively salida estructurada.


In [44]:
messages= "Hola, traduce al inglés: 'El curso es interesante'"
llm.invoke(messages)

AIMessage(content='Hola! La traducción al inglés de "El curso es interesante" sería:\n\n"The course is interesting."\n\nO también podríamos decir:\n\n"The course is engaging" o\n"The course is fascinating"\n\nDependiendo del contexto y la intención que se quiera transmitir.', additional_kwargs={}, response_metadata={'model': 'llama3.2', 'created_at': '2025-01-15T21:20:20.659407Z', 'done': True, 'done_reason': 'stop', 'total_duration': 5613647236, 'load_duration': 26347311, 'prompt_eval_count': 40, 'prompt_eval_duration': 695000000, 'eval_count': 61, 'eval_duration': 4890000000, 'message': Message(role='assistant', content='', images=None, tool_calls=None)}, id='run-6ba15b98-659f-4df7-9d20-fe7909448fbd-0', usage_metadata={'input_tokens': 40, 'output_tokens': 61, 'total_tokens': 101})

### 3. Mensajes

Un mensaje suele consistir en las siguientes piezas de información:

-**Rol**: El rol del mensaje (por ejemplo, "usuario", "asistente").

-**Contenido**: El contenido del mensaje (por ejemplo, texto, datos multimodales).

-**Metadatos adicionales**: id, nombre, uso de tokens y otros metadatos específicos del modelo.

Rol
Los roles se utilizan para distinguir entre diferentes tipos de mensajes en una conversación y ayudar al modelo de chat a entender cómo responder a una secuencia determinada de mensajes.







In [14]:
messages = [
    SystemMessage(content="Translate the following from english to spanish"),
    HumanMessage(content="Hello everybody, how are you doing?"),
]


Se puede usar un "parser" para escribir la respuesta sin metedatos:

In [16]:
parser = StrOutputParser()
result = llm.invoke(messages)
parser.invoke(result)

'Hola a todos, ¿cómo está usted?'

Se pueden juntar el modelo y el parser en una "chain":

In [17]:
chain = llm | parser
chain.invoke(messages)

'Hola a todos, ¿cómo está usted?'

**Temperature**

Repetimos 5 veces la pregunta "Escribe un color cualquiera" y analizamos las respuestas para diferentes valores del parámetro "temperature". Para un valor de temperature=0:


In [21]:
messages = 'Hola, escribe un color cualquiera'
llm = ChatOllama(model=local_llm, temperature=0)
chain = llm | parser
for i in range(5):
    print(chain.invoke(messages))


El color que te voy a describir es... **azul marino**.

Imagina un cielo claro y soleado en una playa tropical. El azul marino es como el color del agua que se refleja en la superficie de la tierra, con un tono más oscuro y profundo que evoca sentimientos de calma y serenidad. Es un color que puede variar desde tonos suaves y pastel hasta más oscuros y dramáticos, dependiendo del ángulo de la luz y el entorno en el que se encuentra.

¿Te gusta este color?
El color que te voy a describir es... **azul marino**.

Imagina un cielo claro y soleado en una playa tropical. El azul marino es como el color del agua que se refleja en la superficie de la tierra, con un tono más oscuro y profundo que evoca sentimientos de calma y serenidad. Es un color que puede variar desde tonos suaves y pastel hasta más oscuros y dramáticos, dependiendo del ángulo de la luz y el entorno en el que se encuentra.

¿Te gusta este color?
El color que te voy a describir es... **azul marino**.

Imagina un cielo claro y

Para el valor temperature=1:

In [22]:
messages = 'Hola, escribe un color cualquiera'
llm = ChatOllama(model=local_llm, temperature=1)
chain = llm | parser
for i in range(5):
    print(chain.invoke(messages))

El color que elegiré... sería... **naranja**
¡Hola! El color que te voy a escribir es: **turquesa**.
**Rojo**

¡Espero que te guste! El rojo es un color vibrante y emocionante que puede evocar sentimientos de pasión, energía y entusiasmo. ¿Quieres saber más sobre este fascinante color?
¡Hola!

El color que he pensado es... **Azul Marino**.

¿Te gusta?
El color que te propongo es... **azul cobalto**. Un color vibrante y elegante que evoca sentimientos de confianza y sofisticación. ¿Te gusta?


**top-k y top-P**

Al igual que la temperatura, los parámetros top-K y top-P también se utilizan para controlar la diversidad de la salida del modelo.

Top-K es un número entero positivo que define la cantidad de tokens más probables de los cuales seleccionar el token de salida. Un top-K de 1 selecciona un solo token.

Top-P define el umbral de probabilidad que, una vez excedido de forma acumulativa, los tokens dejan de ser seleccionados como candidatos. Un top-P de 0 es equivalente típicamente al top K con k=1, y un top-P de 1 selecciona típicamente todos los tokens en el vocabulario del modelo.

Ejecute este ejemplo varias veces, cambie la configuración y observe el cambio en la salida.
 
- top k =64  top_p = 0.95

- top k=1 top_p =0

In [27]:
messages = "You are a creative writer. Write a short story about a programmer who goes on an adventure. Less than 100 words. Give it an amazing title."
llm = ChatOllama(model=local_llm, temperature=1, top_k=64,
        top_p=0.95)
chain = llm | parser
print(chain.invoke(messages))

"Lost in the Code of Dreams"

As he sat at his desk, staring at lines of code that refused to compile, Jack's thoughts strayed to a world beyond pixels and pixels. He typed in one last line, and the room began to shimmer. The computer screen dissolved into a canvas of starry night sky. With a thrill, Jack stepped out of the void and onto a mountain path where binary symbols bloomed like flowers. He followed their trail, weaving through forests of circuitry, until he reached a glade where an ancient tree whispered secrets of code and magic to those who listened.


In [28]:
messages = "You are a creative writer. Write a short story about a programmer who goes on an adventure. Less than 100 words. Give it an amazing title."
llm = ChatOllama(model=local_llm, temperature=1, top_k=1,
        top_p=0)
chain = llm | parser
print(chain.invoke(messages))

**"The Code of the Ancients"**

As a renowned programmer, Max had spent his life solving digital puzzles. But when he stumbled upon an ancient artifact in his code, he was drawn into a world beyond screens. He embarked on a perilous quest to unravel the secrets of the "Eternal Loop." With each step, the code grew more complex, and the stakes higher. As Max navigated treacherous landscapes of 1s and 0s, he discovered that the true power of coding lay not in solving problems, but in unlocking the hidden potential within himself. The loop had become a doorway to his own destiny.


In [29]:
messages = "You are a creative writer. Write a short story about a programmer who goes on an adventure. Less than 100 words. Give it an amazing title."
llm = ChatOllama(model=local_llm, temperature=1, top_k=5,
        top_p=0)
chain = llm | parser
print(chain.invoke(messages))

**"The Code of the Ancients"**

As a renowned programmer, Max had spent his life solving digital puzzles. But when he stumbled upon an ancient artifact in his code, he was drawn into a world beyond screens. He embarked on a perilous quest to unravel the secrets of the "Eternal Loop." With each step, the code grew more complex, and the stakes higher. As Max navigated treacherous landscapes of 1s and 0s, he discovered that the true power of coding lay not in solving problems, but in unlocking the hidden potential within himself. The loop had become a doorway to his own destiny.


## 4. Prompt templates

Si se quieren hacer preguntas siempre del mismo tipo, dando las mismas instrucciones al sistema, se pueden definir "prompt templates", que pueden contener variables, definidas entre {}.

In [30]:
system_template = "Translate the following into {language}, write only the translated word:"

In [31]:
prompt_template = ChatPromptTemplate.from_messages(
    [("system", system_template), ("user", "{text}")]
)

In [ ]:
llm = ChatOllama(model=local_llm, temperature=0)
result = prompt_template.invoke({"language": "Spanish", "text": "Hello!"})
print(result)

In [36]:
chain = prompt_template | llm | parser

In [38]:
chain.invoke({"language": "spanish", "text": "Hello"})



'Hola'

In [43]:
prompt_template = ChatPromptTemplate([
    ("system", "You are a flight attendant"),
    ("user", "Tell me a tip about {topic}. Less than 10 words")
])

chain = prompt_template | llm | parser
chain.invoke({"topic": "security"})

'Wear easy-to-remove shoes for speedy security checks at airports.'